#### PRNG evolution analysis with prediction attacks

The cell below contains experimental configuration settings in the form of a python dictionary.

In [1]:
from Core.Experiment import Experiment

Using TensorFlow backend.


In [3]:
from Core.PrngMethods import Get_Defs

defs = Get_Defs()

print('Prng defs: ',defs)
'''
for key, val in defs.items():    
    configuration = {
        'SEED_METHOD' : 'ticks',               
        'PRNG_METHOD' : key,       
        'NUM_SETS' : 100,                     
        'SET_LEN': 1000,                       
        'VERBOSE' : 0,
        'IS_NEW_MODEL' : True,
        'PATH' : './' + key + '/',                              
        'BATCH_SIZE': 15,
        'NUM_EPOCHS': 20,
        'OPTIMIZER_METHOD' : 'adam',
        'LOSS_METHOD':'mean_squared_error',
        'LOUD_LOGGING': False
    }  

    E = Experiment(configuration)
    E.perform()
    '''
configuration = {
        'SEED_METHOD' : 'ticks',               
        'PRNG_METHOD' : 'Linear_Congruential',       
        'NUM_SETS' : 1000,                     
        'SET_LEN': 2000,                       
        'VERBOSE' : 1,
        'IS_NEW_MODEL' : True,
        'PATH' : './' + 'Linear_Congruential' + '/',                              
        'BATCH_SIZE': 15,
        'NUM_EPOCHS': 5000,
        'OPTIMIZER_METHOD' : 'adam',
        'LOSS_METHOD':'mean_squared_error',
        'LOUD_LOGGING': False,
        'LR': .001
    }  

E = Experiment(configuration)
E.perform()

Prng defs:  {'Middle_Square': '1946', 'Linear_Congruential': '1958', 'Lagged_Fibonacci': '1958', 'Wichmann_Hill': '1982', 'Park_Miller': '1988', 'Maximally_Periodic_Reciprocals': '1992', 'Mersenne_Twister': '1998'}

**********PRNG_METHOD: Linear_Congruential running**********
Epoch 1/5000
 780/1000 [======================>.......] - ETA: 0s - loss: 3.0411e-04

KeyboardInterrupt: 

In [31]:
#Seed generator tests.

from datetime import *

def ticks():
    return  str( int( (datetime.utcnow() - datetime(1, 1, 1)).total_seconds() * 10000000 ))[-6:]

print(int(ticks()))

862656
